# Collect validation results for all methods, months, and regions, then summarize with various univariate and multivariate metrics

In [1]:
import sys

sys.path.insert(0, "../source")

import numpy as np
import pandas as pd

from stat_tools import collect_metrics, multivariate_dss


In [2]:
blocks = ["b1", "b2"]
months = ["02", "04", "07", "10"]

validation_results = []
for block in blocks:
    for month in months:
        df = pd.read_csv(
            f"../data/production/models/2021{month}/validation_results_{block}.csv", index_col=0
        )
        validation_results.append(df)

df_results = pd.concat(validation_results, axis=0).reset_index(drop=True)
df_results


,Method,Month,Region,lat,lon,data,predictions,mspe,difference,ratio,covered,INT,DSS
0,Trend surface,202102,b1,40.025,-94.825,-0.232284,0.257301,0.199556,-0.489585,-1.095962,True,1.754201,-0.410528
1,Trend surface,202102,b1,40.025,-94.775,-0.010717,0.258267,0.203467,-0.268984,-0.596320,True,1.771309,-1.236654
2,Trend surface,202102,b1,40.025,-93.275,0.456127,0.274542,0.159274,0.181586,0.454998,True,1.567183,-1.630106
3,Trend surface,202102,b1,40.025,-93.225,-0.078473,0.274478,0.181601,-0.352951,-0.828237,True,1.673426,-1.019966
4,Trend surface,202102,b1,40.025,-91.825,1.154629,0.265364,0.348070,0.889265,1.507294,True,2.316757,1.216584
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15784,Cokriging,202110,b2,41.425,-101.375,0.418129,0.295172,0.115791,0.122958,0.361341,True,1.336164,-2.025400
15785,Cokriging,202110,b2,41.475,-102.975,-0.184186,0.219189,0.106699,-0.403375,-1.234893,True,1.282631,-0.712786
15786,Cokriging,202110,b2,41.475,-102.925,-0.059710,0.218412,0.106036,-0.278122,-0.854099,True,1.278643,-1.514489
15787,Cokriging,202110,b2,41.475,-101.425,0.112492,0.291260,0.106672,-0.178767,-0.547348,True,1.282470,-1.938408


In [3]:
df_metrics_sub = collect_metrics(df_results).reset_index()
df_metrics_sub


,Method,Month,Region,N,BIAS,RASPE,DSS_MEAN,INT_MEAN,Coverage
0,Cokriging,202102,b1,685,0.116,0.514,-0.337,2.859,0.964
1,Cokriging,202102,b2,554,-0.046,0.268,-1.421,1.709,0.989
2,Cokriging,202104,b1,563,0.005,0.371,-1.034,1.967,0.972
3,Cokriging,202104,b2,594,-0.104,0.369,-0.965,1.964,0.980
4,Cokriging,202107,b1,757,0.058,0.583,-0.134,2.850,0.950
5,Cokriging,202107,b2,712,-0.014,0.563,-0.220,2.950,0.944
6,Cokriging,202110,b1,690,-0.032,0.320,-1.344,1.645,0.962
7,Cokriging,202110,b2,708,-0.055,0.289,-1.444,1.552,0.982
8,Kriging,202102,b1,685,0.118,0.514,-0.336,2.861,0.964
9,Kriging,202102,b2,554,-0.044,0.267,-1.422,1.710,0.989


In [4]:
multivariate_scores = []
for names, df_group in df_results.groupby(["Method", "Month", "Region"]):
    method, month, block = names
    method = method.split(" ")[0].lower()

    data = np.expand_dims(df_group["data"].values, axis=-1)
    mean = pd.read_csv(
        f"../data/production/models/{month}/mean_{method}_{block}.csv", header=None
    ).values
    covariance = pd.read_csv(
        f"../data/production/models/{month}/covariance_{method}_{block}.csv", header=None
    ).values
    mDSS = np.round_(multivariate_dss(data, mean, covariance), 3)
    df = pd.DataFrame(
        dict(Method=[method], Month=[month], Region=[block], mDSS=[mDSS]), index=[1]
    )
    multivariate_scores.append(df)

df_mDSS = pd.concat(multivariate_scores, axis=0).reset_index(drop=True)
df_mDSS.loc[:, "Method"] = df_mDSS.loc[:, "Method"].str.capitalize()
df_mDSS.loc[df_mDSS["Method"] == "Trend", "Method"] = "Trend surface"
df_mDSS


,Method,Month,Region,mDSS
0,Cokriging,202102,b1,-286.355
1,Cokriging,202102,b2,-823.766
2,Cokriging,202104,b1,-633.871
3,Cokriging,202104,b2,-664.242
4,Cokriging,202107,b1,-309.920
5,Cokriging,202107,b2,-356.546
6,Cokriging,202110,b1,-1078.974
7,Cokriging,202110,b2,-1105.997
8,Kriging,202102,b1,-286.075
9,Kriging,202102,b2,-823.726


In [16]:
df_metrics = df_metrics_sub.merge(df_mDSS, on=["Method", "Month", "Region"]).rename(
    columns={"INT_MEAN": "INT", "DSS_MEAN": "DSS", "mDSS": "MDSS"}
)
df_metrics["Month"] = df_metrics["Month"].apply(str)

# df_metrics.loc[:, "Month"] = df_metrics["Month"].apply(
#     lambda yearmonth: "-".join([str(yearmonth)[:4], str(yearmonth)[4:]])
# )

metrics_keep = ["BIAS", "RASPE", "INT", "DSS", "MDSS"]


def format_metrics(df: pd.DataFrame, month: str, metrics: list) -> pd.DataFrame:
    df = df[df["Month"] == month].drop(columns="Month")
    df = (
        pd.melt(df, id_vars=["Method", "Region"], var_name="Metric")
        .pivot(index=["Region", "Method"], columns="Metric", values="value")
        .rename(index={"b1": "Corn Belt", "b2": "Cropland"})
    )
    df.columns.name = None
    return df.loc[:, metrics]


# df_metrics_b1 = format_metrics_region(df_metrics, "b1", metrics_keep)
# df_metrics_b2 = format_metrics_region(df_metrics, "b2", metrics_keep)
df_metrics_july = format_metrics(df_metrics, "202107", metrics_keep)


In [17]:
df_metrics_july


BIAS  RASPE    INT    DSS     MDSS
Region    Method                                            
Corn Belt Cokriging      0.058  0.583  2.850 -0.134 -309.920
          Kriging        0.007  0.585  2.928 -0.120 -306.667
          Trend surface -0.028  0.617  3.024  0.028   21.083
Cropland  Cokriging     -0.014  0.563  2.950 -0.220 -356.546
          Kriging       -0.034  0.564  2.925 -0.206 -355.893
          Trend surface -0.066  0.599  3.046 -0.082  -58.063

In [22]:
def metrics_to_latex(df: pd.DataFrame, month: str):
    bold_min = lambda col: ["font-weight:bold" if x == col.min() else "" for x in col]
    bold_abs_min = lambda col: [
        "font-weight:bold" if np.abs(x) == np.abs(col).min() else "" for x in col
    ]
    table_tex = (
        df.style.apply(bold_abs_min, subset=pd.IndexSlice[["Corn Belt"], ["BIAS"]])
        .apply(bold_min, subset=pd.IndexSlice[["Corn Belt"], ["RASPE"]])
        .apply(bold_min, subset=pd.IndexSlice[["Corn Belt"], ["INT"]])
        .apply(bold_min, subset=pd.IndexSlice[["Corn Belt"], ["DSS"]])
        .apply(bold_min, subset=pd.IndexSlice[["Corn Belt"], ["MDSS"]])
        .apply(bold_abs_min, subset=pd.IndexSlice[["Cropland"], ["BIAS"]])
        .apply(bold_min, subset=pd.IndexSlice[["Cropland"], ["RASPE"]])
        .apply(bold_min, subset=pd.IndexSlice[["Cropland"], ["INT"]])
        .apply(bold_min, subset=pd.IndexSlice[["Cropland"], ["DSS"]])
        .apply(bold_min, subset=pd.IndexSlice[["Cropland"], ["MDSS"]])
        .format(precision=3, escape="latex")
    ).to_latex(
        position="h!",
        label=f"tab:cv-metrics-{month}",
        caption="Caption here",
        sparse_columns=True,
        multirow_align="c",
        clines="skip-last;data",
        hrules=True,
        convert_css=True,
    )
    print(table_tex)
    return table_tex


In [23]:
display(df_metrics_july)
table_b1 = metrics_to_latex(df_metrics_july, "202107")


BIAS  RASPE    INT    DSS     MDSS
Region    Method                                            
Corn Belt Cokriging      0.058  0.583  2.850 -0.134 -309.920
          Kriging        0.007  0.585  2.928 -0.120 -306.667
          Trend surface -0.028  0.617  3.024  0.028   21.083
Cropland  Cokriging     -0.014  0.563  2.950 -0.220 -356.546
          Kriging       -0.034  0.564  2.925 -0.206 -355.893
          Trend surface -0.066  0.599  3.046 -0.082  -58.063

\begin{table}[h!]
\caption{Caption here}
\label{tab:cv-metrics-202107}
\begin{tabular}{llrrrrr}
\toprule
 &  & BIAS & RASPE & INT & DSS & MDSS \\
Region & Method &  &  &  &  &  \\
\midrule
\multirow[c]{3}{*}{Corn Belt} & Cokriging & 0.058 & \bfseries 0.583 & \bfseries 2.850 & \bfseries -0.134 & \bfseries -309.920 \\
 & Kriging & \bfseries 0.007 & 0.585 & 2.928 & -0.120 & -306.667 \\
 & Trend surface & -0.028 & 0.617 & 3.024 & 0.028 & 21.083 \\
\cline{1-7}
\multirow[c]{3}{*}{Cropland} & Cokriging & \bfseries -0.014 & \bfseries 0.563 & 2.950 & \bfseries -0.220 & \bfseries -356.546 \\
 & Kriging & -0.034 & 0.564 & \bfseries 2.925 & -0.206 & -355.893 \\
 & Trend surface & -0.066 & 0.599 & 3.046 & -0.082 & -58.063 \\
\cline{1-7}
\bottomrule
\end{tabular}
\end{table}



In [8]:
# display(df_metrics_b2)
# table_b2 = metrics_to_latex(df_metrics_b2, "b2")
